## **Yisheng - A natural healing doctor using RAG**
This is the example of how an LLM can be used together with the custom data to answer specialized questions related to a field of interest. In this example, we'll minimize the use of higher level methods and libraries to show the basic steps involved in the RAG. I have used the Google Gemini API as it offers generous free-tier for the developers unlike OpenAI.

### 1- Import the Libraries and load the environment variables
First code block is to supress the warning messages. Then we import the libraries and load the GOOGLE_API_KEY from .env file.   
 
*remember to add your .env to the .gitignore*

In [32]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [9]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.document_loaders.json_loader import JSONLoader
from langchain import PromptTemplate
from langchain_community.vectorstores.faiss import FAISS
import os
from dotenv import load_dotenv
import json
import pandas as pd

DEBUG = False

load_dotenv()

if DEBUG:
    os.environ["GOOGLE_API_KEY"]



### 2- Create the LLM Object
We'll create 2 models   

1. The generative/Chat model for the response creation
2. The embedding model to vectorize the retrieval data

In [6]:
# the generative model for the response creation. You can adjust the temperature to control the randomness of the response  
llm = GoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
    )

# the embeddings model for the vectorization of the data
embeddings_model = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
    )

if DEBUG:
    print(llm.invoke("Tell me a joke"))

### 3- Load the JSON data file
A simple json data file with ailments and home remedies data. Load the data as the ground truth for the response to queries

In [8]:
# Function to extract the page_content from Document object and convert to list of strings
def extract_text_column(documents, column_name):
    text_list = []
    for doc in documents:
        text_list.append(doc.page_content)
    return text_list

# Load the data from the json file
dataloader = JSONLoader(
    file_path='remedies2.jsonl',
    jq_schema='.Remedy',
    text_content=False,
    json_lines=True
    )

data = dataloader.load()

# convert to text list for the vectorization
text_list = extract_text_column(data, 'page_content')

if DEBUG:
    print(text_list)



### 4- Vectorize the data and store in Vector Database    
Now it's time to vectorize the data and store in the vector database. I have picked FAISS for the convenience. You can select other databases

In [11]:
if DEBUG:
    embeddings = embeddings_model.embed_documents(text_list)
    print("Embed documents:")
    print(f"Number of vector: {len(embeddings)}; Dimension of each vector: {len(embeddings[0])}")

# Create the vector store
vector_store = FAISS.from_texts(text_list,embeddings_model)


### 5- Take the user input and query the vector store
Take the user input and perform similarity search on the vector store to retrieve matching records

In [ ]:
#get your questions
user_question = input("Please enter your question")
match = vector_store.similarity_search(user_question)
print(match[0].page_content)


The remedies for Cold are Ginger tea and Cloves


### 6- Generate the human-like response using the LLM
And finally we'll generate the friendly response by engineering the prompt and invoking LLM

In [16]:


# Create the prompt text
remedy_template = """You are a traditional medic who advises based on natural healing methods and home remedies. \
You help people understanding the natural healing methods based on the expert advice given to you

Here is the expert advise for you:
{advice}"""


# Create the prompt template with a variable input
prompt_template = PromptTemplate(template=remedy_template)


# Function to generate a prompt with the given input
def generate_prompt(advice):
    return prompt_template.format(advice=advice)

# Generate the prompt
prompt = generate_prompt(match[0].page_content)

# Invoke Google Gemini for the response
response = llm.invoke(prompt)

print(response)

Ah, a nasty cold has taken hold, has it?  Don't you worry, my dear.  Nature provides many gentle remedies to soothe those sniffles and chase away the chills.  The wise ones, and I count myself among them, have long known the power of ginger and cloves in combating the common cold.

Let's talk about these two potent allies:

**Ginger:** This warming root is a true marvel. It possesses potent anti-inflammatory properties, helping to reduce the congestion that often accompanies a cold.  The warmth itself can be incredibly soothing, easing the aches and pains that often accompany a cold.

* **How to use it:** I recommend a simple ginger tea.  Take a knob of fresh ginger – about an inch or two – and finely grate it.  Steep this in a cup of boiling water for 10-15 minutes. You can add a touch of honey or lemon for extra sweetness and vitamin C, but avoid dairy as it can increase mucus production. Sip this warm tea throughout the day.  The spiciness will clear your sinuses and the warmth will